In [69]:
"""
For this experiment  you need to have your own csv containing types of questions that users can
ask or ask the AI team to for their dataset as we did not upload the csv to this repo
"""

'\nFor this experiment  you need to have your own csv containing types of questions that users can\nask or ask the AI team to for their dataset as we did not upload the csv to this repo\n'

In [88]:
import pandas as pd
from ai_docs_filterer_for_RAG import run_rm_labeller
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openai import OpenAIProvider
from openai import AsyncAzureOpenAI
import os
import asyncio
from dotenv import load_dotenv
from ccs_website_data import  fetch_all_ccs_frameworks


load_dotenv()

True

In [135]:
test_data = pd.read_csv("query_filter_questions.csv")
ccs_frameworks = fetch_all_ccs_frameworks()
pydantic_azure_client = AsyncAzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_deployment=os.getenv("DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),

)
pydantic_rm_labeller_model = OpenAIChatModel(
    model_name= os.getenv("DEPLOYMENT_NAME"),
    provider=OpenAIProvider(openai_client=pydantic_azure_client),

)

rm_descriptions = "\n".join([
    f"RM: {r.rm_number} | "
    f"Keywords: {r.keywords if 'keywords' in r and str(r.keywords).strip() else 'N/A'} | "
    f"Summary: {r.summary} | "
    f"Pillar: {r.pillar} ({r.category})"
    for _, r in ccs_frameworks.iterrows()
])


Starting to fetch and clean frameworks (Status: Live,Expired)...
Fetched Page 1. Total frameworks collected: 268
Finished. Reached the last page: 1.

✅ DataFrame created successfully!


## Running experiment

In [128]:

async def process_dataframe_sequentially(df):
    results = []

    for index, row in df.iterrows():
        # Using .get() or index-based access to be safe
        query_text = row['query']
        print(f"Processing row {index + 1}/{len(df)}...")

        try:
            # 1. DIRECT AWAIT: No asyncio.run() needed here
            result = await run_rm_labeller(
                pydantic_rm_labeller_model,
                rm_descriptions,
                query_text
            )
            results.append(result.rm_number)

            # 2. STEADY PACE: A small pause to keep Azure happy
            await asyncio.sleep(0.5)

        except Exception as e:
            print(f"Error on row {index}: {e}")
            results.append(f"Error: {type(e).__name__}")

    df['predicted_rm'] = results
    return df

# Run it like this in your notebook cell:


In [129]:
result_df = await process_dataframe_sequentially(test_data)

Processing row 1/95...
Processing row 2/95...
Processing row 3/95...
Processing row 4/95...
Processing row 5/95...
Processing row 6/95...
Processing row 7/95...
Processing row 8/95...
Processing row 9/95...
Processing row 10/95...
Processing row 11/95...
Processing row 12/95...
Processing row 13/95...
Processing row 14/95...
Processing row 15/95...
Processing row 16/95...
Processing row 17/95...
Processing row 18/95...
Processing row 19/95...
Processing row 20/95...
Processing row 21/95...
Processing row 22/95...
Processing row 23/95...
Processing row 24/95...
Processing row 25/95...
Processing row 26/95...
Processing row 27/95...
Processing row 28/95...
Processing row 29/95...
Processing row 30/95...
Processing row 31/95...
Processing row 32/95...
Processing row 33/95...
Processing row 34/95...
Processing row 35/95...
Processing row 36/95...
Processing row 37/95...
Processing row 38/95...
Processing row 39/95...
Processing row 40/95...
Processing row 41/95...
Processing row 42/95...
P

In [136]:

accuracy = (result_df['Truth Set'] == result_df['predicted_rm']).mean()

print(f"Overall Accuracy: {accuracy * 100:.2f}%")

Overall Accuracy: 77.89%


In [137]:
mismatched_df = result_df[result_df['Truth Set'] != result_df['predicted_rm']]
print(len(mismatched_df))

21


In [138]:
# result_df.to_csv("filter_llm_results.csv", index=False)